### Satellite Collector Sandbox to build functions

In [3]:
from json import load
from datetime import date
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelhub.geo_utils import pixel_to_utm, utm_to_pixel
import pandas as pd

In [4]:
geojson_file = "../data/target_zone.geojson"

with open("../secrets/sentinel_api_credentials.json", "r") as infile:
    credentials = load(infile)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

In [22]:
def satellite_query(start_date, end_date, max_cloud_per, location, auth):
    """Query satellite image from Sentinel-2 API.
    
    Retrieves image ids from Sentinel-2 taking a date range, max cloud
    percentage, location coordinates in geojson and authentication details.
    
    Args:
        start_date: date in the date class format.
        end_date: date in the date class format.
        max_cloud_per: integer for the max percentage of cloud coverage.
        location: geojson file with target coordinates.
        auth: authentication object (SentinelAPI)
    
    Returns:
        Pandas dataframe containing a list of images for manual selection 
        and download.
    """
    shape = geojson_to_wkt(read_geojson(location))

    images = auth.query(
        shape,
        date=(start_date, end_date),
        platformname="Sentinel-2",
        processinglevel = "Level-2A",
        cloudcoverpercentage=(0, max_cloud_per)
    )

    images_down = api.to_dataframe(images)
    images_df = images_down[['summary', 'beginposition','cloudcoverpercentage','vegetationpercentage',
                           'waterpercentage','size']]
    return images_df


In [7]:
def sat_downloader(images_dataframe):
    """Download the products from satellite-query output.
    
    Filter all image uuid from dataframe all downloads the whole product
    package.
    
    Args:
        images_dataframe: pandas dataframe output from satellite-query output.
    
    Returns:
        Satellite files saved in data/products folder.
    """
    uuid_list = images_dataframe.index
    api.download_all(uuid_list, directory_path='../data/products/')

In [23]:
date_init = date(2023, 1, 1)
date_end = date(2023, 2, 28)

In [24]:
test_output = satellite_query(date_init, date_end, 80, geojson_file, api)

In [25]:
test_output

,summary,beginposition,cloudcoverpercentage,vegetationpercentage,waterpercentage,size
26f7977e-c883-4e35-95a5-28e288dec401,"Date: 2023-02-25T13:12:49.025Z, Instrument: MS...",2023-02-25 13:12:49.025,31.682077,52.720696,4.737068,1.07 GB
6e250784-1bf2-4228-b239-fb38ff72df58,"Date: 2023-02-25T13:12:49.025Z, Instrument: MS...",2023-02-25 13:12:49.025,21.921374,16.573086,59.846675,957.29 MB
0011833b-0a34-4cd7-8d9a-b70f3e60fab7,"Date: 2023-02-20T13:12:41.024Z, Instrument: MS...",2023-02-20 13:12:41.024,18.997638,65.461481,4.964187,1.10 GB
8034c420-8787-4724-bfbf-8f19e05be55b,"Date: 2023-02-20T13:12:41.024Z, Instrument: MS...",2023-02-20 13:12:41.024,35.713518,10.006231,51.341945,1010.76 MB
0982d219-c32b-4d96-b733-a67e90e7a433,"Date: 2023-02-15T13:12:39.024Z, Instrument: MS...",2023-02-15 13:12:39.024,37.356758,7.746364,53.181380,915.34 MB
5f6baae7-01f9-4b6b-a0e2-863b9aa8150e,"Date: 2023-02-15T13:12:39.024Z, Instrument: MS...",2023-02-15 13:12:39.024,47.847635,40.459311,1.199263,1.03 GB
47d40891-8846-4934-8aec-12325467c1dc,"Date: 2023-02-10T13:12:41.024Z, Instrument: MS...",2023-02-10 13:12:41.024,51.617217,37.312025,1.179024,1.06 GB
da65f5a8-8355-448b-889f-0b64a3406762,"Date: 2023-02-10T13:12:41.024Z, Instrument: MS...",2023-02-10 13:12:41.024,76.746237,6.969263,15.563864,918.45 MB
fa46c0e8-1c29-4d3e-849b-b94aec0e7b6f,"Date: 2023-02-05T13:12:39.024Z, Instrument: MS...",2023-02-05 13:12:39.024,61.043298,27.322176,1.184206,1.04 GB
27785e85-049c-4249-ba3b-2701ce21921d,"Date: 2023-02-05T13:12:39.024Z, Instrument: MS...",2023-02-05 13:12:39.024,71.359646,4.482613,23.103082,944.52 MB
